# Data Preparation and Transformation with IPUMS USA

## Introduction

This notebook is designed to guide you through a data cleaning workflow using 2010 ACS data previously downloaded from the [IPUMS USA](https://cps.ipums.org/usa) project using the [ipumsr R package](https://cran.r-project.org/web/packages/ipumsr/index.htm).

This notebook is designed to be run directly after working through the **IPUMS ACS Data Extraction** notebook.  If you are working on this notebook without previously runnig the **IPUMS ACS Data Extraction** notebook, you will need to copy the "IPUMS_ACS_5y_2022.rds" file into your working directory prior to running this notebook.

Users are also encouraged to revise this notebook for their own purposes and using their own data.

By working through this notebook, you will learn some common workflows for cleanig and recoding population data.

### ★ Prerequisites ★
* Complete Chapter 1.1: Introduction to IPUMS and the IPUMS API
* Set Up Your [IPUMS Account and API Key](https://account.ipums.org/api_keys)
* Complete Chapter 1.2: IPUMS USA Data Extraction Using ipumsr

At the end of Chapter 1.2: IPUMS USA Data Extraction Using ipumsr, you saved your data extraction as two file formats *ipums_usa_example.rds* and *ipums_usa_example.csv*.  You will need these files to run this notebook.  If you are working throuhg this chapter without previously completing, Chapter 1.2, you will need to copy the "IPUMS_ACS_5y_2022.rds" file into your working directory prior to running this notebook.

#### Overview
This notebook includes the following sections:

1. Setup
2. Initial Review
3. Cleaning and Recoding Continuous Variables
5. Cleaning and Recoding Categorical Variables
6. Final Review

## 1. Setup

Before running this script, you will need to install and load the following packages into your R environment:

[**dplyr**](https://cran.r-project.org/web/packages/dplyr/index.html) A Grammar of Data Manipulation. This notebook uses the the following functions from *dplyr*.

* [*mutate*](https://rdrr.io/cran/dplyr/man/mutate.html) · create, modify, and delete columns
* [*case_when*](https://rdrr.io/cran/dplyr/man/case_when.html) · a general vectorized if-else
* This notebook also uses [*%>%*](https://magrittr.tidyverse.org/reference/pipe.html), referred to as the *pipe* operator, which is used to pass the output from one function directly into the next function for the purpose of creating streamlined workflows.  The *pipe* operator is a commonly used component of the [*tidyverse*](https://www.tidyverse.org).

[**haven**](https://cran.r-project.org/web/packages/haven/index.html) Import and Export 'SPSS', 'Stata' and 'SAS' Files.  This notebook uses the the following functions from *haven*.

* [*as_factor()*](https://rdrr.io/cran/haven/man/as_factor.html) · for formating categorical variables as factors.

### 1a. Install and Load Required Packages
If you have not already installed the required packages, uncomment and run the code below:

In [9]:
# install.packages(c("dplyr", "haven"))

Load the packages into your workspace.

In [2]:
library(dplyr)
library(haven)


Attaching package: 'dplyr'


The following objects are masked from 'package:stats':

    filter, lag


The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union




### 1b. Read the Data File

Run the following line of code to read in the *ipums_usa_example.rds* file into memory.  You may need to update the file path to reflect the file's location on your machine or in your working directory.

The *ipums_usa_example.rds* file contains information from the 2010 ACS.

In [4]:
dat <- readRDS("ipums_usa_example.rds")

As a refresher, let's take a look at the number of observations and the variables in the data.

In [5]:
dim(dat)

[1] 3061692      18

The data includes information on 18 variables for 3,061,692 observations.

In [7]:
colnames(dat)

[1] "YEAR"     "SAMPLE"   "SERIAL"   "CBSERIAL" "HHWT"     "CLUSTER" 
 [7] "PUMA"     "STRATA"   "GQ"       "PERNUM"   "PERWT"    "SEX"     
[13] "AGE"      "RACE"     "RACED"    "EDUC"     "EDUCD"    "INCTOT"

Below is a referece list of the variables included in this data extraction.

**Variable Selection**
* Public Use Microdata Area (PUMA)
* Sex (SEX)
* Age (AGE)
* Race (RACE)
* Educational Attainment (EDUC)
* Total Personal Income (INCTOT)

**Detailed Supplements for Selected Variables**
* Race (detailed) (RACED)
* Education (detailed) (EDUCD)

**IPUMS Preselected Variables**
* Census Year (YEAR)
* IPUMS Sample Identifier (SAMPLE)
* Household Serial Number (SERIAL)
* Original Census Bureau Household Serial Number (CBSERIAL)
* Household Weight (HHWT)
* Household Cluster for Vaccine Estimation (CLUSTER)
* Household Strata for Variance Estimation (STRATA)
* Group Quarters Status (GQ)
* Person Number in Sample Unit (PERNUM)
* Person Weight (PERWT)

In [6]:
head(dat)

YEAR,SAMPLE,SERIAL,CBSERIAL,HHWT,CLUSTER,PUMA,STRATA,GQ,PERNUM,PERWT,SEX,AGE,RACE,RACED,EDUC,EDUCD,INCTOT
<int>,<int+lbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl+lbl>,<dbl>,<int+lbl>,<dbl>,<dbl>,<int+lbl>,<int+lbl>,<int+lbl>,<int+lbl>,<int+lbl>,<int+lbl>,<dbl+lbl>
2010,201001,1,69,96,2.01e+12,400,40001,1,1,96,2,75,1,100,6,63,7500
2010,201001,2,80,97,2.01e+12,2200,220001,1,1,97,1,25,1,100,11,114,17000
2010,201001,2,80,97,2.01e+12,2200,220001,1,2,128,2,26,1,100,7,71,13000
2010,201001,2,80,97,2.01e+12,2200,220001,1,3,182,1,3,1,100,0,2,9999999
2010,201001,3,140,90,2.01e+12,100,10001,1,1,90,2,87,1,100,7,71,29400
2010,201001,4,224,82,2.01e+12,1300,130001,1,1,82,1,33,1,100,10,101,28000


Notice that this data is in *[tibble](https://tibble.tidyverse.org)* format rather than the more common *data.frame* format you might be used if you are a regular R user.  A tibble can be thought of as a version of a data.frame that includes additional functionality and metadata visibility.  It is also more compatible with [*tidyverse*](https://www.tidyverse.org) packages, including the [*dplyr*](https://cran.r-project.org/web/packages/dplyr/index.html) package we use in this notebook.

## 2. Review the Data

First, let's start with a quick summary using the *summary* command in base R.  We will restrict the summary view to just the subset of demographic variables since those are the variables we will focus on in the data cleaning process for this exercise.

The *summary* command provides a overview of our data and is a quick and easy way of exploring the structure of our data as well as potential issues.

In [8]:
summary(dat[, c("SEX", "AGE", "RACE", "RACED", "EDUC", "EDUCD", "INCTOT")])

      SEX             AGE             RACE           RACED      
 Min.   :1.000   Min.   : 0.00   Min.   :1.000   Min.   :100.0  
 1st Qu.:1.000   1st Qu.:19.00   1st Qu.:1.000   1st Qu.:100.0  
 Median :2.000   Median :40.00   Median :1.000   Median :100.0  
 Mean   :1.514   Mean   :39.69   Mean   :1.738   Mean   :175.5  
 3rd Qu.:2.000   3rd Qu.:58.00   3rd Qu.:1.000   3rd Qu.:100.0  
 Max.   :2.000   Max.   :95.00   Max.   :9.000   Max.   :990.0  
      EDUC            EDUCD            INCTOT       
 Min.   : 0.000   Min.   :  1.00   Min.   : -15870  
 1st Qu.: 3.000   1st Qu.: 30.00   1st Qu.:   9600  
 Median : 6.000   Median : 63.00   Median :  30000  
 Mean   : 5.838   Mean   : 60.81   Mean   :1879017  
 3rd Qu.: 8.000   3rd Qu.: 81.00   3rd Qu.:  85000  
 Max.   :11.000   Max.   :116.00   Max.   :9999999  

We can see from this summary that all the variables are coded as numeric, even though many of them are categorical, such as "race" or "worker class".  The numeric values for these categorical variables are useless without data labels or a codebook reference.  Fortunately, the IPUMS extraction and download process included metadata for our data, which includes labels for relevant variables.  We can also refer to the [IPUMS website](https://usa.ipums.org/usa-action/variables/group/demog) for detailed information on each variable.  It is a good data stewardship practice to always have the codebook or other reference material nearby while performing the data cleaning process.

The high-level summary also shows us a few potential issues with these data.

1. **All variables are coded as numeric, even though many of them are actually categorical.**  It is very common to code categorical variables using numeric codes but it is important to remember that these categorical variables are useless without data labels or a codebook reference.  Fortunately, the IPUMS extraction and download process included metadata for our data, which includes labels for relevant variables.  We can also refer to the [IPUMS website](https://usa.ipums.org/usa-action/variables/group/demog) for detailed information on each variable.  It is a good data stewardship practice to always have the codebook or other reference material nearby while performing the data cleaning process.
2. **There are no missing values.**  In the summary view of R, these would be indicated by a count of "NA" values for each of the variables.  While it is possible that our data actually has no missing values, it is also very possible that missing values are coded with a specific coding as mentioned in the section above.  Especailly with very large and complex datasets, we should retain a skepticism about the lack of missing values and review each variable and its corresponding labels and codebook during the cleaning process.
3. **The largest total income (INCTOT) value is 9999999.**  While it is possible that this is the actual maximum total income value in the dataset, it is more likely that this is a topcoded reference value.  Often values in multiples of 9s, e.g. "999" or "9999999" indicate a topcoded value, missing, or other special value.  This quick summary overview reminds us that when we clean the INCTOT variable, we should be doubly sure to reference the codebook and data labels.

In the next two sections we will review each of the demographic variables, keeping these potential issues at the front of our mind as we go.

This process will involve looking for and managing and issues with the data, such as missing values or values which need to be recoded, as well as reorganizing variables to make them easier to use or to suit the needs of our project.

## 3. Clean and Recode Categorical Variables

We will start with the three categorical variables in our set demographic variables.  The categorical variables in our dataset include:

* [sex (SEX)](https://usa.ipums.org/usa-action/variables/SEX#codes_section)
* [race (RACE and RACED)](https://usa.ipums.org/usa-action/variables/RACE#codes_section)
* [educational attainment (EDUC and EDUCD)](https://usa.ipums.org/usa-action/variables/EDUC#codes_section)

To help us easily view the percent of the data which is represented by each variable level - as well as the data labels *{{variable}}*, count of observations *n*, and numeric level indicator *Code* - we will create and use following helper function *calculate_percentage*.  Alternately, we could use the base R function *table* which would provide us with the counts for each variable level but would not show the percents or labels.

In [ ]:
calculate_percentage <- function(data, variable) {
    data %>%
    count({{variable}}) %>%
    mutate(
        Code = row_number(),
        {{variable}} := as_factor({{variable}}),
        Percent = round(n / sum(n) * 100, 2)
    ) %>%
    select({{variable}}, Code, n, Percent)
}

### 3a. Sex

The [IPUMS codebook for SEX](https://usa.ipums.org/usa-action/variables/SEX#codes_section) shows us that there are three categories for this variable.

1. Male
2. Female
9. Missing/blank

Similar to the INCTOT case we talked about in the inital review, the SEX variable also has a special coding using 9s.  A code of 9 for SEX indicates a missing value.

Let's take a look at the breakdown of the SEX variable in our data.

In [ ]:
calculate_percentage(dat, SEX)

All values in the SEX variable in our data are 1 (male) or 2 (female) and there are no missing values.

### 3b. Race

As we remember from our initial *summary* view, the IPUMS API call included both the RACE variable and a supplementary detailed race RACED variable.  The [IPUMS codebook for RACE and RACED](https://usa.ipums.org/usa-action/variables/RACE#codes_section) shows us that there are nine categories for the basic race variable (RACE) and 253 categories for the detailed race variable (RACED).

Let's first take a look at the RACE variable.

In [ ]:
calculate_percentage(dat, RACE)

As we saw in the codebook, the RACE variable has nine categories, and our dataset indcludes individuals in each of the nine categories.

**Note:** The acronym NEC, as in the "Other race, nec" category, stands for "not elsewhere classified" and is commonly used in demographic data.

The RACE variable is an example of a very common occurrance in demographic data where one or a few categories represent the vast the majority of the data and the remaining categories are relatively rare.  In our sample over 70% of the individuals are White!  This sometimes poses an issue in analysis, especially if we are interested in focusing on minority populations.  We won't worry about it much here, but it is something to think about when you conduct your own analyses.

We know that the detailed race variable (RACED) has a lot of categories (253).  Let's check how many of them are represented in our data.

In [ ]:
length(unique((dat$RACED)))

The individuals in our data represent 137 of the possible 253 race categories.

137 is still a lot of categories so we aren't going to attempt to view the full table for this variable.  Instead, let's take a look at the first ten categories.

In [ ]:
head(calculate_percentage(dat, RACED), 10)

We can already see that this version of the race variable has a lot of additional information.  The first two categories (White and Black/African-American) are the same as the RACE variable but the next few categories split up the more general "American Indian or Alaska Native" cateogry and specify specific Native American tribes.  Without looking at the entire list of all categories in RACED we can get an idea of the level of detail included in this variable.

This level of detail might come in handy later on if we use this data for a detailed race-based analysis.  But for now let's keep it simple and stick to just using the RACE variable.

#### Recoding the Race Variable

Even the 9 race categories in the simple RACE variable is a lot, and if we are plannign to run a regression or do some other analysis where statistical power is important, we might want to simplify this variable even further.

Depending on the type of analysis you conduct, it might be essential to reduce the number of cateogies - especially condensing categories with very few observations - so that you can run an analysis with sufficient statisitcal power.  When it comes to a variable like race, the practice of combining categories can be contentious and nuanced.  For this exercise, we will condense our data to five very commonly used race categories for working with data on the United States general population.  These are the categories currently in use for the U.S. Decennial Census.

1. White
2. Black or African-American
3. American Indian or Alaska native
4. Asian or Pacific Islander
5. Some Other Race
6. Two or More Races

This process will condense our data down from 9 to 6 categories.  Individuals classified as "4. Chinese", "5. Japanese", "6, Other Asian or Pacific Islander" will be reclassified to the new "4. Asian or Pacific Islander" category and indiviuals classified as "8. Two major races" and "9. Three or more major races" will be reclassified the new "6. Two or More Races" category.

We will use the *mutate()* and *case_when()* functions from the *dplyr* package to create a new variable (RACE_RECODE) based on information from the existing RACE variable.

In [ ]:
dat <- dat %>%
  mutate(RACE_RECODE = case_when(
    RACE == 1 ~ 1,
    RACE == 2 ~ 2,
    RACE == 3 ~ 3,
    RACE %in% c(4, 5, 6) ~ 4,
    RACE == 7 ~ 5,
    RACE %in% c(8, 9) ~ 6))

We should also specify the new RACE_RECODE variable as a [**factor**](https://r4ds.hadley.nz/factors.html) which a data type used to represent categorical variables that have a limited and fixed set of possible categories.  The individual categories of a factor are usually referred to as "levels".  A nice feature of factors is that we can also set the order of the levels and add labels to each level.

The RACE_RECODE only has five possible levels, corresponding to the five race groupings we created, so it fits the critera of a factor.    We will add labels in this step which will help us remembewhat races are grouped into the categories represented by r each of t RACE_RECODE levels.e.

In [ ]:
dat <- dat %>%
  mutate(RACE_RECODE = factor(RACE_RECODE,
                              levels = c(1, 2, 3, 4, 5, 6),
                              labels = c("White",
                                         "Black",
                                         "American Indian or Native American",
                                         "Asian or Pacific Islander",
                                         "Other Race",
                                         "Two or More Races")))

Let's take a look at the new variable.

In [ ]:
calculate_percentage(dat, RACE_RECODE)

We now have a new, simplified version of the RACE variable with fewer categories.  The smallest group (American Indian or Native American) corresponds to 1.18% of the population while the largest group (White) corresponds to 71.19% of the population.

### 3c. Education

Similar to the RACE variable, the eductional attainment variable EDUC also has a supplementary, more detailed educational attainment variable EDUCD.  The EDUC and EDUCD variables provide information on total individual educational attainment.  The [IPUMS codebook for EDUC and EDUCD](https://usa.ipums.org/usa-action/variables/EDUCD#codes_section) shows us that there are 13 categories for the basic education variable (EDUC) and 44 categories for the detailed education variable (EDUCD).

Let's first take a look at the EDUC variable.

In [ ]:
calculate_percentage(dat, EDUC)

Taking a look at the EDUC variable, we can see that the first level corresponds to either "N/A" or "no schooling".  At the beginning of the EDA section, we mentioned the possibility of some variables having levels which correspond to missing or not applicable values.  Unfortunatly for us, the two components of the "N/A or no schooling" group correspond to two very different situations.  Most likely, we would want to seperate these two situations, recoding the "N/A" values as "missing" and either creating a seperate group for individuals with no schooling or combining them into a "little or no schooling" category.

We can also see that, while the EDUC variable provides information on the number of years of education completed, it does not provide information on degree completion.  In most cases, there is a significant social and economic difference between actually receiving a high school diploma and completing 12 years of education but not receiving a high school diploma.  It would be best if we could explore additional detail for the individuals in some of these categories.

Fortunatly, we have the EDUCD variable which has a lot more information on educational attainment.  Let's take a look at what information is available in EDUCD.

In [ ]:
calculate_percentage(dat, EDUCD)

The EDUCD variable has many more categories including seperate categories for "N/A" and "no schooling completed" as well as detailed information on degree completion!

#### Recoding the Education Variable

Like with the RACE variable in the previous section, we are going to recode the education variable into a new set of groups which might be a bit more useful for analysis.  This time we will use information from the EDUCD variable since it contains the additional detail we need to be able to make groups for "N/A", "no schooling", and completion of specific degrees.

Our recode will have the following seven categories.

1. eight or fewer years of eduction (including no schooling)
2. some high school (9-12 years, no degree)
3. high school diploma, GED, or alternative credential completed
4. some college (1+ years, no degree)
5. associate's degree
6. bachelor's degree
7. advanced degree (master's, professional, or doctoral degree)

This new variable will include missing values since we plan to recode instances of "N/A" to missing.  In R, missing values are represented by the special object *NA*.  Most analytical functions in R automatically ignore *NA* values or have an option to ignore them so recoding missing categories as *NA* will make things easier for us down the line.

In [ ]:
dat <- dat %>%
  mutate(EDUC_RECODE = case_when(
    EDUCD == 1 ~ NA,                # N/A
    EDUCD %in% c(2:26) ~ 1,         # eight or fewer years (including no schooling)
    EDUCD %in% c(30:61) ~ 2,        # some high school (9-12 years, no degree)
    EDUCD %in% c(63:64) ~ 3,        # high school diploma, GED, or alternative credential completed
    EDUCD %in% c(65:71) ~ 4,        # some college (1+ years, no degree)
    EDUCD == 81 ~ 5,                # associate's degree
    EDUCD == 101 ~ 6,               # bachelor's degree
    EDUCD %in% c(114:116) ~ 7))     # advanced degree (master's, professional, or doctoral degree)

As with the RACE_RECODE variable in the previous section, we will classify the EDUC_RECODE as a factor and attach category labels.

In [ ]:
dat <- dat %>%
  mutate(EDUC_RECODE = factor(EDUC_RECODE,
                              levels = c(1, 2, 3, 4, 5, 6, 7),
                              labels = c("eight or fewer years",
                                         "some high school, no degree",
                                         "high school degree or equivalent",
                                         "some college, no degree",
                                         "associate's degree",
                                         "bachelor's degree",
                                         "advanced degree")))

Let's take a look at the new variable.

In [ ]:
calculate_percentage(dat, EDUC_RECODE)

Again we have a new, simplified version of the a categorical variable with fewer categories.  In this case, the new variable is both simplier than either of the original RACE and RACED variables and also reorganizes the data in a way which is more meaningful for the analyses we have in mind.

## 4. Clean and Recode Continuious Variables

Now that we have finished reviewing the three categorical demographic varables, we will move on to the two continuous demographic variables.  The continuous variables in our dataset include:

* [age (AGE)](https://usa.ipums.org/usa-action/variables/AGE#codes_section)
* [individual total income (INCTOT)](https://usa.ipums.org/usa-action/variables/INCTOT#codes_section)

### 4a. Age

The AGE variable provides the numeric age, in years, for each indiviual in the sample.  Checking out the [IPUMS codebook for AGE](https://usa.ipums.org/usa-action/variables/AGE#codes_section) tells us that there is a missing value code corresponding to "999" but based on our initial review of the data, we don't appear to have any "999" codes and therefore no missing data for age.  We will double check for missing values in this step.

We could use the *calculate_percentage* function we had been using for the categorical variables to view the AGE variable but that would give us a table for every age in the dataset, probably not very useful.

Instead, let's take a look at the summary statistics using the *summary()* command from base R and distribution for this numeric variable using the historgam *hist()* command also from base R.

Both functions are from base R!  The information and graphics they will provide are relatively simple and not very pretty but the commands are also simple and easy to remember, part of the base R environment (so they don't require any additional packages), and computationally non-intensive.  You could use more complicated functions to generate summary tables and visualizations but at this stage, when we just want quick snapshots of the data so we can make management decisions, the simple base R functions work great.

In [ ]:
summary(dat$AGE)

Ages in this sample range from 0 to 97 which seems reasonable.  And we also can confirm that we don't have any instances of the "999" code indicating missing data.

In [ ]:
hist(dat$AGE, breaks = 50, main = "Histogram of Age", xlab = "Age")

The distribution of ages, as shown in the histogram, also seems to be a reasonable representation of a random sample of the population.

While the numeric AGE variable is very useful, it likely contains more detail that we will need for our analysis.  So let's create a new categorical age variable based on age ranges.  Our new variable will use the following commonly used age ranges for demographic and sociological research.

1. 0 to 17 (children)
2. 18 to 24 (young adults)
3. 25 to 44
4. 45 to 64
5. 65+ (older adults; retirement age)

We will use the *mutate()* function from the *dplyr* package to recode information from AGE variable into a new AGE_CAT variable.

In [ ]:
dat <- dat %>%
  mutate(AGE_CAT = case_when(
    AGE < 18 ~ 1,
    AGE %in% c(18:24) ~ 2,
    AGE %in% c(25:44) ~ 3,
    AGE %in% c(45:64) ~ 4,
    AGE >= 65 ~ 5))

Although the original AGE variable was continuous, the new AGE_CAT variable is categorical and like RACE_RECODE and EDUC_RECODE, AGE_CAT is also a factor.  Therefore, we will specify AGE_CAT as a factor and set labels for each variable level using the same workflow we used for the cateogorical variables.

In [ ]:
dat <- dat %>%
  mutate(AGE_CAT = factor(AGE_CAT,
                              levels = c(1, 2, 3, 4, 5),
                              labels = c("under 18",
                                         "18-24",
                                         "25-44",
                                         "45-64",
                                         "65+")))

Now that we have a labeled, categorical version of the AGE variable, let's take a look at the percent breakdown by age range.

In [ ]:
calculate_percentage(dat, AGE_CAT)

The new AGE_CAT variable simplifies the AGE information into a format which is likely easier for us to use and understand.

### 4b. Income

The last demographic varaible we will work with is the income variable (INCTOT).  Like the age variable, the original version of this variable is numeric and specifes the exact income for each surveyed individual.  We can take a look at the 

As with AGE, rather than using the *calculate_percentage* function, we'll use the base R *summary()* function to check out the summary statistics for the variable, and the base R *hist()* function to view the distribution of income.

In [ ]:
summary(dat$INCTOT)

The "9999999" maximum value should raise some alarm and remind you that we previously noticed the maximum INCTOT value is "9999999".  Something to be suspicious about...

In [ ]:
hist(dat$INCTOT, breaks = 100, main = "Histogram of Total Income", xlab = "Total Income")

The histogram showing the distribution of income values is another red flag that something is not right.  Numeric, continuous demographic variables almost always follow some type of normal or log-normal distribution.  However, there is clearly a large chunk of the data categorized at a very high value.  An anomaly which doesn't seem to follow the pattern of the rest of the data.

The solution is to take a look at the [IPUMS codebook for INCTOT](https://usa.ipums.org/usa-action/variables/INCTOT#codes_section).  The codebooks tells us that value "9999998" indicates "unknown" and the value "9999999" indictes "N/A".

So for our next step we should recode these two values as *NA* so they won't be included in any calculations or in while visualizing the distribution.

In [ ]:
dat <- dat %>%
  mutate(INCTOT = na_if(INCTOT, 9999998),
         INCTOT = na_if(INCTOT, 9999999))

Let's take another look at the summary statistics and the distribution of values after recoding the missing information.

In [ ]:
summary(dat$INCTOT)

In [ ]:
hist(dat$INCTOT, breaks = 100, main = "Histogram of Total Income", xlab = "Total Income")

After the recode the data summary and distribution appear much more as we would expect.  The distribution shows that the majority of the individuals in the sample have incomes around 200,000 or less but there are a few outliers ranging up to the largest income in the sample, 1.87 million.

Similar to age, total income is not very useful as a raw number.  Instead, it's often more useful to look at income brackets rather than raw income values.

A recode would be especially useful in our situation since we have an income distribution with a very long right tail, indicating that we have a few individuals with very high incomes who are skewing the overall distribution.  When we have a highly skewed dataset, values usually take on wildly different meanings at different parts of the distribution.  For example, the difference between an income of 10,000 and 50,000 corresponds to a very large difference in lifestyle and socioeconmic status.  However, the difference between an income of 1,010,000 and 1,050,000 is likely negligible, both very high incomes correspond to a similair lifestyle and very high socioeconomic status.

We will recode the income variable in the following way:

1. < 25,000 (low income)
2. 25,000 to 49,999 (lower-middle income)
3. 50,000 to 74,999 (middle income)
4. 75,000 to 149,999 (upper-middle income)
5. 150,000 or more (high income)

In [ ]:
dat <- dat %>%
  mutate(INCTOT_CAT = case_when(
    INCTOT < 25000 ~ 1,
    INCTOT %in% c(25000:49999) ~ 2,
    INCTOT %in% c(50000:74999) ~ 3,
    INCTOT %in% c(75000:149999) ~ 4,
    INCTOT >= 150000 ~ 5))

As usuall, we will code this variable as a factor and create a set of labels for each level.

In [ ]:
dat <- dat %>%
  mutate(INCTOT_CAT = factor(INCTOT_CAT,
                              levels = c(1, 2, 3, 4, 5),
                              labels = c("under 25,000",
                                         "25,000 to 49,999",
                                         "50,000 to 74,999",
                                         "75,000 to 149,999",
                                         "150,000 or more")))

Let's take a look at the percentage breakdown for our new categorical variable.

In [ ]:
calculate_percentage(dat, INCTOT_CAT)

As with AGE_CAT, the INCTOT_CAT variable simplifies the income information into a simplier format which is easier to understand and work with.

## 5. Final Review

We've completed initial data management and recode steps for each of the demographic variables we requested in our abstract.  Let's take a final look at the updated summary information including all the new demographic varaibles we created in this section.

In [ ]:
summary(dat[, c("SEX", "AGE", "AGE_CAT", "RACE", "RACED", "RACE_RECODE", "EDUC", "EDUCD", "EDUC_RECODE", "INCTOT", "INCTOT_CAT")])

We've made a lot of updates!  We now have categorical versions of our two continuous numeric variables, AGE and INCTOT, and new, recoded versions of RACE and EDUC.  We've also recoded the missing values in EDUC_RECODE and INC_TOT to *NA* so we can use these variables in analyses without having to worry about incorrectly treating the missing codes like real numers.

Let's make a simplified version of the data, specifically for analysis, that only includes the variables we plan to keep.

This step isn't essential but it will give us a smaller version of the data to work with in the next steps of our analysis which will be less confusing and less computationally intensive to work with.

Specifically, we will restrict our data to the following demographic variables:

1. sex (SEX)
2. age (AGE)
3. age categories (AGE_CAT)
4. race categories (RACE_RECODE)
5. educational attainment categories (EDUC_RECODE)
6. income (INCTOT)
7. income categories (INCTOT_CAT)

And the two geographic identification variables we included in our original data extraction:

6. state FIPS code (STATEFIP)
7. county FIPS code (COUNTYFIP)

In [ ]:
dat_analysis <- dat %>% select(STATEFIP, COUNTYFIP, SEX, AGE, AGE_CAT, RACE_RECODE, EDUC_RECODE, INCTOT, INCTOT_CAT)

Let's take a look at the first few lines of *dat_analysis*.

In [ ]:
head(dat_analysis)

And a look at the summary information for *dat_analysis*.

In [ ]:
summary(dat_analysis)

We'll save a copy of *dat_analysis* in the .rds file format using the *saveRDS* base R function.  The .rds format will allow us to retain the labels and metadata in our dataset for the next time we want to import the same file back into R.  One downside to the .rds format is it is only useable within R.

Alternatively we could save this file as a .csv or other common format which would allow us to easily open it in other settings.  However, for this exercise, we are simply interested in storing a copy of our data so that we can use it in another notebook withouth having to go through the data cleaning and labeling steps again.

In [10]:
saveRDS(dat_analysis, "ipums_usa_analysis.rds")

ERROR: Error in eval(expr, envir, enclos): object 'dat_analysis' not found


We now have a cleaned, quality dataset saved and ready for our next step, exploratory data analysis (EDA)!

## Recommended Next Steps

* **Continue with Chapter 3: Data Cleaning and Preparation**
  * 3.2: Spatial Data Preparation and Transformation with IPUMS NHGIS
* **Move on to Chapter 4: Exploratory Data Analysis (EDA)**
  * 4.1: Exploratory Data Analysis (EDA) with IPUMS USA